In [1]:
import azureml.core
print(azureml.core.VERSION)

1.0.8


In [2]:
# Create a new workspace if needed, takes about 2 minutes
from azureml.core import Workspace
ws = Workspace.create(name='rlab314ws',
                      subscription_id='37bede89-47b0-42cb-94c0-173753dcad79', 
                      resource_group='rlab314wsrg',
                      create_resource_group=True,
                      location='eastus2' # Or other supported Azure region   
                     )

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BELJ7SASN to authenticate.


Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.


In [5]:
# Save the config for reuse later
ws.write_config()

Wrote the config file config.json to: /home/labadmin/aml_config/config.json


In [6]:
# Connect to existing workspace
from azureml.core import Workspace
ws=Workspace.from_config()

Found the config file in: /home/labadmin/aml_config/config.json


In [7]:
# Declare dependencies
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("numpy")
myenv.add_conda_package("pillow")
myenv.add_conda_package("tensorflow")
myenv.add_conda_package("matplotlib")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
#with open("myenv.yml","r") as f:
#    print(f.read())

In [11]:
# Register a trained model
from azureml.core.model import Model

model = Model.register(model_path = "tfeval",
                       model_name = "icondetection",
                       description = "icon detection model using TF",
                       workspace = ws)

print(model.version)

Registering model icondetection


In [9]:
# Create and register an image
from azureml.core.image import ContainerImage

# Image configuration
image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                 runtime = "python",
                                                 conda_file = "myenv.yml",
                                                 description = "icon detection model image using TF",
                                                 )

In [10]:
# Register the image from the image configuration in ACR - takes 3-5 mins
image = ContainerImage.create(name = "icondetection", 
                              models = [model], #this is the model object
                              image_config = image_config,
                              workspace = ws
                              )

image.wait_for_creation(show_output=True)

Creating image
Running.........................................................
SucceededImage creation operation finished for image icondetection:1, operation "Succeeded"


In [ ]:
# Get the logs if needed
#print(image.image_build_log_uri)

In [12]:
# Deploy to ACI - takes 3-4 mins
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 2, 
                                               memory_gb = 2, 
                                               tags = {"data": "icon detection web service", "type": "icon detection"}, 
                                               description = 'TF Object Detection model for icon detection')

from azureml.core.webservice import Webservice

service_name = 'aci-id-1'
service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                            image = image,
                                            name = service_name,
                                            workspace = ws)
service.wait_for_deployment(show_output = True)

Creating service
Running....................................
SucceededACI service creation operation finished, operation "Succeeded"


In [13]:
print(service.state)
#service.get_logs()

Healthy


In [14]:
# Get the scorig URI
service.scoring_uri

'http://52.151.224.120:80/score'

In [15]:
# Call the endpoint
import requests
import json
import numpy as np

input_data = "{\"data\": [\"some data\"]}"

headers = {'Content-Type':'application/json'}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("prediction:", resp.text)

POST to url http://52.151.224.120:80/score
prediction: <html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.10.3 (Ubuntu)</center>
</body>
</html>

